### downloading the passengers data

In [40]:
!wget -nc https://lazyprogrammer.me/course_files/airline_passengers.csv

File ‘airline_passengers.csv’ already there; not retrieving.



### updating the statsmodels api

In [41]:
!pip install -U statsmodels

### loading the libraris

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing


### loading the data fromcsv

In [43]:
df = pd.read_csv("airline_passengers.csv",index_col = 'Month',parse_dates = True)

In [44]:
df.head()

,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


### making the index of data frequency aware as Month start

In [45]:
df.index.freq = 'MS'

In [46]:
df.head()

,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [47]:
df.index.freq

<MonthBegin>

In [48]:
df.index.inferred_freq

'MS'

In [49]:
df.shape

(144, 1)

### setting the important parameters for our task

In [50]:
# As sume the forecast horizon we care about is 12
# Validate over 10 steps

h = 12
steps = 10 # The number of test forecasts (or validation steps) you plan to generate in your walk-forward process.For instance, if steps=10, you’re going to make 10 separate forecasts, each shifted in time, to evaluate your model.

# Below is the constraint has to be always satisfied
# (Ntest + (steps - 1)) + h <= T - 1
Ntest = len(df)-h-steps + 1

# Lets understand why this makes sense

#Suppose len(df) = 100.
# h = 12 (you need 12 future points after you start predicting).
# steps = 10 (you plan to make 10 forecast runs, each one step apart).

# Ntest = 100 - 12 - 10 + 1
# Ntest = 79

#This means your first forecast start point is at index 79. Why?

#The first forecast will use indices [79 ... 79+(h-1)=79+11=90] for validation.
#The last (10th) forecast will start at index 79+(10-1)=79+9=88, and it needs data through index 88+(12-1)=99 for validation, which is exactly the last point in the dataset.




This means we start our first test forecast at index 79.

Why index 79?
You want to make steps = 10 forecasts, each forecast needing h = 12 points for validation. Each new forecast will start one step later in the time series.

First forecast: starts at index 79
You need 12 data points starting from this position. Let’s count 12 points starting at 79:
79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90
That’s exactly 12 points (count them: from 79 to 90 inclusive is 12 steps).
So for the first forecast, you’ll use indices 79 through 90 to validate your predictions.

Second forecast: starts at index 80 (one step after 79)
Similarly, it will need 12 points: 80 through 91 (80,81,82,83,84,85,86,87,88,89,90,91).

...Continue this process until the 10th forecast:
The 10th forecast starts at index 79 + (10 - 1) = 79 + 9 = 88.
From index 88, you again need 12 points: 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99.
These 12 points end exactly at index 99, which is the last data point you have. Perfect fit!



Why does this work out so neatly?

Starting at 79 for the first forecast ensures that after making the first forecast (which uses up to index 90), you can still move forward and make 9 more forecasts, each shifted by one step and each requiring 12 points.
By the time you get to the 10th forecast (at index 88), you still have enough room (up to index 99) to get those 12 validation points.
If you started any later than 79, you wouldn’t have enough data at the end.
If you started any earlier, you could still do it, but 79 is the exact point calculated by the formula that ensures no data is wasted.

In [51]:
## configuration hyperparameterrs to try

trend_type_list = ["add","mul"]
seasonal_type_list = ["add","mul"]
damped_trend_list = [True,False]
init_method_list = ["estimated","heuristic","legacy-heuristic"]
use_boxcox_list = [True,False,0]

**For damped_trend_list:**

True: If damped_trend is True, the trend component will be allowed to grow or decline at a slower rate as time goes on. This avoids projections that become too large or too negative far out into the future.

False: If damped_trend is False, any trend in the data is modeled as ongoing at a constant rate. If the model identifies an upward trend, it continues projecting growth at that same rate indefinitely, which might not always be realistic.

**For init_method_list :**

What is initialization in ETS models?
When fitting an ETS model (Exponential, Trend, Seasonal model), you need initial values for the model’s components (level, trend, seasonal factors) before iterative optimization begins. How you pick these initial values can affect model convergence, speed, and quality of results.

"estimated":
Under the "estimated" initialization method, the initial values of the level, trend, and seasonal components are treated as parameters to be optimized by the fitting procedure. In other words, the model will internally attempt to find the best starting points that minimize the error on the training data. This often yields more accurate forecasts since it tailors the initial state specifically to your data, but it can be slower since you are adding more parameters to estimate.

"heuristic":
The "heuristic" method uses a predefined formula or set of rules to guess the initial values. This approach does not treat initialization as part of the optimization problem. Instead, it uses simple statistics (like the average for level, a difference-based method for trend, and some pattern extraction for seasonality) to set starting points. This is usually faster but might be less accurate than "estimated," because the initial values might not be perfectly aligned with your particular dataset.

"legacy-heuristic":
"legacy-heuristic" refers to the initialization method that older versions of the model or library used by default. It’s similar in spirit to "heuristic" but uses older rules or formulas. It’s typically included for backward compatibility, ensuring that if you rely on older results or code, you can replicate them. The logic might be slightly different or simpler than the current heuristic method.

Why does it make sense?
Different initialization strategies balance complexity and speed. If you’re after the most accurate model and have time to let the optimization run, "estimated" might be best. If you need quick results, "heuristic" methods give you a reasonable starting point. If you need to reproduce old results, "legacy-heuristic" can help you do so.


**use_boxcox_list = [True, False, 0]**

What is Box-Cox transformation?
A Box-Cox transformation is a way to transform non-stationary or heteroscedastic time series into a more "normal" looking form. It often helps stabilize variance and make the time series more suitable for modeling. ETS models assume more stable patterns if the data’s variance does not change drastically over time.

True: If use_boxcox is True (without specifying a parameter), the model will attempt to automatically find a Box-Cox transformation lambda parameter that best stabilizes the variance.
False: If use_boxcox is False, no Box-Cox transformation is applied. The model will work on the raw data, which is simpler but may lead to less reliable forecasts if the variance changes over time.
0: If you explicitly set use_boxcox = 0, this is equivalent to applying a log transformation (a common special case of Box-Cox with lambda=0). It’s a straightforward way to stabilize variance if your data is strictly positive and multiplicative patterns are suspected.
Why does it make sense?
If your data grows in amplitude as it grows in mean (for example, bigger sales months also have more volatility), applying a Box-Cox transformation can make the model’s job easier, often resulting in more accurate forecasts.

### Defining the walkforward function

In [60]:
def walkforward(
    trend_type,
    seasonal_type,
    damped_trend,
    init_method,
    use_boxcox,
    debug = False
  ):

    ## store errors
    errors = []
    seen_last  = False
    steps_completed = 0


    for end_of_train in range(Ntest,len(df)-h + 1):

      # we don't have to manually 'add" the data to our dataset
      # Just index it at the right points - this is a view not a copy
      # so it doesn't take up any extra space or computation

      train = df.iloc[:end_of_train]
      test = df.iloc[end_of_train:end_of_train+h]

      if test.index[-1] == df.index[-1]:
        seen_last = True

      steps_completed += 1

      hw = ExponentialSmoothing(
          train["Passengers"],
          initialization_method = init_method,
          trend = trend_type,
          damped_trend = damped_trend,
          seasonal = seasonal_type,
          seasonal_periods = 12,
          use_boxcox = use_boxcox,

      )

      res_hw = hw.fit()

      ## Compute the error for the forecast horizon

      fcast  = res_hw.forecast(h)
      error = mean_squared_error(test["Passengers"],fcast)
      errors.append(error)


    if debug:
      print("seen_last:" ,seen_last)
      print("steps completed:",steps_completed)


    return np.mean(errors)

In [61]:
## test our function
walkforward('add', 'add',False,'legacy-heuristic',0,debug = True)

seen_last: True
steps completed: 10


1448.5344452151644

In [62]:
# iterate through all the possible options (i.e grid search)

tuple_of_options_list = (
    trend_type_list,
    seasonal_type_list,
    damped_trend_list,
    init_method_list,
    use_boxcox_list
)

for x in itertools.product(*tuple_of_options_list):
  print(x)

('add', 'add', True, 'estimated', True)
('add', 'add', True, 'estimated', False)
('add', 'add', True, 'estimated', 0)
('add', 'add', True, 'heuristic', True)
('add', 'add', True, 'heuristic', False)
('add', 'add', True, 'heuristic', 0)
('add', 'add', True, 'legacy-heuristic', True)
('add', 'add', True, 'legacy-heuristic', False)
('add', 'add', True, 'legacy-heuristic', 0)
('add', 'add', False, 'estimated', True)
('add', 'add', False, 'estimated', False)
('add', 'add', False, 'estimated', 0)
('add', 'add', False, 'heuristic', True)
('add', 'add', False, 'heuristic', False)
('add', 'add', False, 'heuristic', 0)
('add', 'add', False, 'legacy-heuristic', True)
('add', 'add', False, 'legacy-heuristic', False)
('add', 'add', False, 'legacy-heuristic', 0)
('add', 'mul', True, 'estimated', True)
('add', 'mul', True, 'estimated', False)
('add', 'mul', True, 'estimated', 0)
('add', 'mul', True, 'heuristic', True)
('add', 'mul', True, 'heuristic', False)
('add', 'mul', True, 'heuristic', 0)
('add

# integrating the walkforward function with the grid search to find the best set of hypermeters

In [ ]:
best_score = float('inf')
best_options = None

for x in itertools.product(*tuple_of_options_list):

  score = walkforward(*x)

  if score < best_score:

    print("Best Score so far",score)
    best_score = score
    best_options = x

Best Score so far 412.81721214194295
Best Score so far 397.5872085746867
Best Score so far 368.78749934592304
Best Score so far 320.6641194372547
Best Score so far 308.1361170888028


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow 

Best Score so far 305.6593349312611
Best Score so far 299.82220927850994
Best Score so far 261.8795073548492
Best Score so far 249.57507607273482


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow 

In [ ]:
print("best_score:",best_score)

In [ ]:
# upacking the best options

trend_type,seasonal_type,seasonal_type,init_method,use_boxcox = best_options

print("trend_type:",trend_type)
print("seasonal_type:",seasonal_type)
print("seasonal_type:",seasonal_type)
print("init_method",init_method)
print("use_boxcox",use_boxcox)